In [15]:
import src.utils as utils
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler

from IPython.display import display
from slugify import slugify
from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
from huggingface_hub import create_repo
from IPython.display import display_markdown
from IPython.display import clear_output
from IPython.utils import capture
from pathlib import Path
import shutil

import time
import os

In [9]:
utils.setup_workspace(config_file="src/configs/workspace_config.yaml")

=== Workspace Setup Complete ===


In [5]:
!accelerate launch src/train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir="./weights/" \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=13278 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=1000 \
  --save_interval=250 \
  --save_sample_prompt=f"concept art of {INSTANCE_NAME} {CLASS_NAME} as star wars character with sci-fi city in background, highly detailed, 8k, uhd, studio lighting, beautiful, high exposure, high contrast" \
  --concepts_list="src/configs/concepts_list.json"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/paths.py:27: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/us

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = "weights/1000/"

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")

g_cuda = torch.Generator(device='cuda')


In [82]:
prompt = """
3D animated illustration of yusufjkhan1 as young  disney pixar animated character : 4.0 |
big disney eyes : 8.5 |
disney 3D,
big hero 6, up,
young pixar,
colorful cartoon,
disney animated,
pixar eyes,
yusufjkhan1,
disney,
pixar, 
professional,
bokeh,
disney pixar
"""

prompt = """
yusufjkhan1 as a marble sculpture : 3.0 |
background of museum : 1.5 |
roman sculpture,
muscular,
muscles,
robes,
marble,
marble statue,
stone sculpture
"""

prompt = """
acrylic painting of yusufjkhan1 as a nomadic desert wanderer : 3.0 |
painting,
high definition,
art,
arab,
bedouin,
desert,
camel,
tribe
"""

prompt = """
colorful animated art of yusufjkhan1 as super saiyan from dragonball z : 3.0 |
dragon ball z,
yusufjkhan1,
UHD,
8k resolution,
goku,
vegeta,
super saiyan,
anime
"""

prompt = """
3D concept art of yusufjkhan1 as kratos from god of war : 3.0 |
yusufjkhan1,
ghost of sparta,
play station,
god of war
"""

prompt = """
animated drawing of yusufjkhan1 as character in Pirates of the Caribbean : 3.0 |
with a pirate ship in the background : 2.0 |
pirates of the caribbean,
jack sparrow,
pirates,
captain barbossa
"""

prompt = """
cartoon animation of yusufjkhan1 as character from disney movie Aladdin : 3.0 |
disney,
cartoon,
2D,
jaffar,
aladdin,
prince ali,
genie
"""

prompt = """
cartoon animation yusufjkhan1 as an anime character from my hero academia : 4.0 |
boku no hero academia, anime, brown skin, yusufjkhan1
"""

prompt = """
cartoon animation of yusufjkhan1 as wolverine from the x-men : 3.0 |
x men,
yusufjkhan1,
wolverine,
highly detailed,
high exposure
"""

negative_prompt = "duplication, smile"

In [ ]:
seed = 52364
g_cuda.manual_seed(seed)
num_samples = 6
guidance_scale = 12
num_inference_steps = 100
height = 512 
width = 512 

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)

  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
config = utils._get_config(utils.CONFIG_FILE)

In [20]:
ckpt_path = Path(config['OUTPUT_DIR']).joinpath('1000', "/model.ckpt")
model_path = Path(config['OUTPUT_DIR']).joinpath('1000')

!python src/convert_diffusers_to_original_stable_diffusion.py --model_path $model_path --checkpoint_path $ckpt_path

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
